# Introduction

### Requirements
#### Globus Transfer Client
 * git clone https://github.com/globusonline/transfer-api-client-python.git
 * python setup.py install

#### Create a Globus Account
* https://www.globus.org/SignUp

#### Save Access Token to a File
* curl -k --user &lt;username&gt;  'https://nexus.api.globusonline.org/goauth/token?grant_type=client_credentials'
* Copy the "access_token" from the response into a file "globus-token"

# Background
<img src="./img/globus-transfer.png" width=70%>

# Configuration

Configure the following variables required to use the transfer client: username and the file in which your access token is stored. 
You can then create a Globus Transfer client object to interact with Globus via its REST APIs.

In [ ]:
import json
from globusonline.transfer.api_client import TransferAPIClient, Transfer, Delete, _endpoint_path
username = "USERNAME"
token_file = "PATH_TO_TOKEN"

### Load token and instantiate client

In [ ]:
with open(token_file, 'r') as f:
    globus_token = f.read()

client = TransferAPIClient(None, goauth=globus_token.strip())

# Transfer client interactions

The following steps will show you how to interact with Globus. 

### Find details of an endpoint
All endpoints are uniquely identified by a name. When retrieving information, you can view the details of the endpoint (owner, contact information, name, etc.), the MyProxy and GridFTP server addresses, and configuration.

In [ ]:
code, reason, data = client.endpoint('go#ep1')
print "Name: %s" % data["canonical_name"]

### List endpoints
Globus has over 8000 endpoints, so you might want to use a limit or change the orderby parameter (options include, name, username, expire_time)

In [ ]:
code, reason, data = client.endpoint_list(limit=5, orderby="username")
for endpoint in data["DATA"]:
    print endpoint["canonical_name"]

### Search for an endpoint
Globus now provides full text search over endpoints and their attributes. This provides an easy way to find an endpoint with limited information. 

In [ ]:
code, reason, data = client.get("endpoint_search?filter_fulltext=ep1")
for endpoint in data["DATA"]:
    print endpoint["canonical_name"]

### Find endpoint activation requirements
Before accessing an endpoint, it must be "activated" by authenticating with the endpoint's configured identity provider. Many endpoints can be automatically activated via your Globus identity (e.g., Globus connect personal, shared, and S3 endpoints). Others may require that you authenticate using an OAuth workflow.

In [ ]:
code, reason, data = client.endpoint_activation_requirements('go#ep1')
print "go#ep1 auto_activation_supported: %s" % data["auto_activation_supported"]

code, reason, data = client.endpoint_activation_requirements('ncsa#BlueWaters')
print "ncsa#BlueWaters Oauth_server: %s" % data["oauth_server"]

### Activate an endpoint
Endpoints that support auto activation can be activated with a single call using your Globus identity.

In [ ]:
code, reason, data = client.endpoint_autoactivate('go#ep1', if_expires_in=600)
code, reason, data = client.endpoint_autoactivate('go#ep2', if_expires_in=600)

print "Activation expires: %s" % data["expire_time"]

### Activate an endpoint that doesn't support auto activation
If we try to auto activate an endpoint that does not support autoactivation, Globus will throw an exception.

In [ ]:
try: 
    code, reason, data = client.endpoint_autoactivate('ncsa#BlueWaters', if_expires_in=600)
except Exception, e:
    print "Cant autoactivate this endpoint!" 
    print e

### List the contents of an endpoint
After activating an endpoint, we can now perform actions on it. For example, we can list the contents of a specified directory. 

In [ ]:
code, reason, data = client.endpoint_ls('go#ep1', '/share/godata')
for file_or_dir in data['DATA']:
    print "%s (%s)" % (file_or_dir['name'], file_or_dir['size'])

### Transfer files
Before transferring data, both participating endpoints must be activated. You must then acquire a submission ID and finally submit the transfer by specifying the two endpoints and the source and destination files or directories. 

In [ ]:
label = "My first transfer"

# activate destination endpoint
code, reason, data = client.endpoint_autoactivate('go#ep2', if_expires_in=600)

#get a unique ID for this transfer
code, message, data = client.transfer_submission_id()
submission_id = data["value"]

# start the transfer
transfer = Transfer(submission_id, 'go#ep1', 'go#ep2', label=label)
transfer.add_item('/share/godata/file1.txt', '/~/myfile.txt')
code, reason, data = client.transfer(transfer)
task_id = data["task_id"]

print "Transfer started with ID %s" % task_id

### Check the status of the transfer
Transfer are conducted asynchronously. You can periodically check the status of the task as follows. 

In [ ]:
code, reason, data = client.task(task_id)
print "Status: %s" % data["status"]

### List the contents of the desination endpoint
Once the file transfer is complete, you can perform another endpoint_ls operation to verify the directory's contents.

In [ ]:
code, reason, data = client.endpoint_ls('go#ep2', '/~/')
for file_or_dir in data['DATA']:
    print "%s (%s)" % (file_or_dir['name'], file_or_dir['size'])

### List the events of this transfer
Globus periodically monitors the progress of a transfer. As such, users can review these markers as well as any errors or faults that occured. 

In [ ]:
code, reason, data = client.task_event_list(task_id)
for event in data["DATA"]:
    print event["description"]

### View transfer tasks
You can review current and previous transfer tasks as well as information about each task including request time, completion time, number of faults, bytes transferred. 

In [ ]:
code, reason, data = client.task_list()
for task in data["DATA"]:
    print "label: %s, bytes transferred: %s" % (task["label"], task["bytes_transferred"])


### Create a new folder
Globus supports other actions on endpoints such as mkdir and delete. 

In [ ]:
code, reason, data = client.endpoint_mkdir('go#ep1', '/~/shared_dir/')
print data["message"]

### Create an shared endpoint
Shared endpoints allow users to share directories within endpoints with other Globus users, groups, and publicly. To create a shared endpoint, you must specify a name and the host endpoint and path to be shared. Note: you must be in the tutorial users group to create shared endpoints. 

In [ ]:
code, reason, data = client.shared_endpoint_create('my_shared_endpoint', 'go#ep1', '/~/shared_dir/')
shared_endpoint_name = data["canonical_name"]
print data["message"]

### Set ACLs on the shared enpoint
Access to a shared endpoint can be dynamically set. To do so, specify the user to share with, and the permissions to grant (r or w). 

In [ ]:
data = {"principal_type":"user",
        "principal":"kyle",
        "send_email":False,
        "DATA_TYPE":"access",
        "path":"/",
        "permissions":"r"}
code, reason, data =  client.post(_endpoint_path(shared_endpoint_name) + "/access", json.dumps(data))
print data["message"]

### Set public ACLs on shared endpoint. 
To share with any Globus user you must specify the "all_authenticated_users" principal. 

In [ ]:
data = {"principal_type":"all_authenticated_users",
        "principal":"",
        "DATA_TYPE":"access",
        "path":"/",
        "permissions":"r"}
code, reason, data =  client.post(_endpoint_path(shared_endpoint_name) + "/access", json.dumps(data))
access_id = data["access_id"]
print data["message"]

### Delete an ACL. 
ACLs can be deleted via their IDs. 

In [ ]:
code, reason, data =  client._delete("%s/access/%s" %(_endpoint_path(shared_endpoint_name), (access_id)))
print data["message"]